In [5]:
#import libraries
import random
import sqlite3
from sqlite3 import Error

#define shopping listS
shoppinglist = ['bread','milk','eggs','chicken','beef','pork','tofu','noodles','rice','water','juice','milk','coffee','tea','lettuce','watermelon','bananas','tomatoes','pickles','lemons','peas','cabbage','chili','cashews','edamame','carrots','radish','beer','papertowels','napkins']
len(shoppinglist)

30

In [ ]:
#generate 20 transactions
def generate_transactions(inputlist):
    index = 0
    transactions = []
    while index < 20:
        
        #generate random number of items in each shopping transaction
        transnum = random.randrange(1,20,1)

        #generate transactions using the transnum
        transaction = random.sample(inputlist,transnum)

        #sort alphabetically and append to transactions list
        transaction.sort()
        transactions.append(transaction)

        index +=1
    
    return transactions

#show transactions
transactions = generate_transactions(shoppinglist)
for listitem in range(len(transactions)):
    print(transactions[listitem])
    if 'cabbage' in transactions[listitem]:
        if 'lettuce' in transactions[listitem]:
            print(True)

In [1]:
#create database(s) and relevant tables

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

def create_table(conn,name):
    """ create table from the create_table_statement"""
    try:
        c = conn.cursor()
        c.execute("""CREATE TABLE IF NOT EXISTS %s (item_id text PRIMARY KEY)""" %(name))
        print('Table created')
        conn.commit()
        conn.close()
    except Error as e:
        print(e)
    

def populate_table(conn,tablename,itemname):
    """populate table from populate_statement"""
    try:
        c = conn.cursor()
        c.execute("""INSERT INTO %s (item_id) VALUES (?)""" %(tablename),[itemname])
        print('Table populated')
        conn.commit()
        conn.close()
    except Error as e:
        print(e)


def query(conn,statement):
    try:
        c = conn.cursor()
        c.execute(statement)
        conn.commit()
        row = c.fetchall()
        print(row)
        conn.close()
    except Error as e:
        print(e)
        
def main():
    database = r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db"
    
    index = 0
    #create the 20 transaction tables
    for index in range(len(transactions)):
        #create database connection
        conn = create_connection(database)
        if conn is not None:
            table_name = 'transaction_'+str(index)
            create_table(conn,table_name)
            #populate each table with the items in each respective transaction
            for item in transactions[index]:
                item_name = item
                conn = create_connection(database)
                populate_table(conn,table_name,item_name)
        else:
            print("Cannot connect to database")
            
#test
if __name__ == '__main__':
    #create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db")
    #main()
    statement = "SELECT * FROM transaction_4 ORDER BY item_id ASC"
    #statement = "UPDATE transaction_4 SET item_id = 'pork' WHERE item_id='edamame'"
    query(create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db"),statement)


NameError: name 'Error' is not defined

In [6]:
import numpy as np
import itertools
class associationMining:
    def __init__(self):
        self.conn = create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db")
        self.result_dict = 1
    #get transactions from database in list form
    def getItems(self,conn,transnum):
        try:
            items_list = []
            c = conn.cursor()
            c.execute("SELECT item_id FROM %s ORDER BY item_id ASC"%('transaction_'+str(transnum)))
            transaction = c.fetchall()
            for item in transaction:
                items_list.append(item[0])
            return items_list
            conn.close()
        except Error as e:
            print(e)

    def getTransactions(self,conn,translength):
            transactions = []
            for i in range(translength):
                transaction = self.getItems(conn,i)
                transactions.append(transaction)
            return transactions

    def scan(self,itemset,transactions):
        count = 0
        for i in range(len(transactions)):
            if set(itemset).issubset(transactions[i]):
                count +=1
        return count   

    #find subsets of given size
    def generateGroups(self,input_set,size):
        return list(itertools.combinations(input_set,size))

    ##Always start with n=1 when calling the function to properly do the algorithm
    #discarded will start off as an empty list
    def apriori(self,transactions,transactionList,min_support,n,discarded):
        #generate sets of n and scan, then filter sets that dont meet min_support

        if n < len(transactions) and n > 0:
            #initialize supports count dictionary
            
            if n <= 1:
                supports = {}
                for transaction in range(len(transactionList)):

                    #generate n sized subsets
                    temp_set = self.generateGroups(transactionList[transaction],n)

                    for item in range(len(temp_set)):
                        #scan transactions for the n sized subsets
                        #prevent duplicate searches
                        if item not in supports:
                            if item not in discarded:
                                item_count = self.scan(temp_set[item],transactionList)

                        #check if the support meets the minimum requirements
                            if item_count / len(transactionList) >= min_support:
                                supports[temp_set[item][0]] = item_count
                            elif item_count / len(transactionList) < min_support:
                                #if doesn't meet minimum requirements, will discard
                                if temp_set[item] not in discarded:
                                    discarded.append(temp_set[item])

            elif n > 1:
                supports = {}
                #generate n sized subsets
                temp_set = self.generateGroups(transactions,n)
                for item in range(len(temp_set)):
                    #scan for n sized subsets that meet minimum support
                    if item not in supports:
                        if item not in discarded:
                            item_count = self.scan(temp_set[item],transactionList)
                            #check if the support meets the minimum requirements
                            if item_count / len(transactionList) >= min_support:
                                supports[temp_set[item]] = item_count
                            elif item_count / len(transactionList) < min_support:
                                #if doesn't meet minimum requirements, will discard
                                if temp_set[item] not in discarded:
                                    discarded.append(temp_set[item])  
                                
            return supports
            #stops the algorithm on an empty supports dictionary
            #if supports:
                #apriori(list(supports.keys()),getTransactions(conn,20),min_support,n+1,discarded)

            #else:
                #print(supports)
                #print('Apriori stopped at iteration {}'.format(n))


    #n=1 when starting the algorithm
    def generateAssociationRules(self,set_info,min_confidence):
        #generate association rules and calculate confidence to compare with min_confidence
        for itemset in range(len(set_info)):
            if set_info[itemset] is not None:
                set_length = len(set_info)
                self.generateGroups()
    
    def algorithm(self,transactionList,min_support,min_confidence,n):
        pass



In [7]:
## Initialize the algorithm

In [8]:
conn = create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db")

a = associationMining()
first_pass = a.apriori(a.getTransactions(conn,20),a.getTransactions(conn,20),0.3,1,[])
second_pass = a.apriori(list(first_pass.keys()),a.getTransactions(conn,20),0.3,2,[])
third_pass = a.apriori(list(second_pass.keys()),a.getTransactions(conn,20),0.3,3,[])
#rule = a.generateAssociationRules(second_pass,.6)

print(first_pass)
print('#'*40)
print(second_pass)
print('#'*40)
print(third_pass)

2.6.0
2.6.0
{'bananas': 6, 'cabbage': 12, 'carrots': 8, 'cashews': 6, 'juice': 6, 'milk': 11, 'pork': 10, 'radish': 6, 'watermelon': 7, 'beer': 8, 'lettuce': 11, 'noodles': 9, 'pickles': 7, 'tea': 10, 'bread': 9, 'lemons': 6, 'napkins': 7, 'papertowels': 8, 'tofu': 6, 'edamame': 6, 'chicken': 9, 'water': 6, 'peas': 9, 'beef': 6, 'eggs': 6}
########################################
{('cabbage', 'carrots'): 6, ('cabbage', 'milk'): 7, ('cabbage', 'beer'): 6, ('cabbage', 'lettuce'): 8, ('cabbage', 'noodles'): 6, ('cabbage', 'tea'): 7, ('cabbage', 'papertowels'): 6, ('cabbage', 'chicken'): 6, ('milk', 'pork'): 6, ('milk', 'lettuce'): 7, ('milk', 'chicken'): 6, ('pork', 'chicken'): 6, ('beer', 'lettuce'): 6, ('beer', 'noodles'): 6, ('lettuce', 'noodles'): 7, ('lettuce', 'chicken'): 6, ('tea', 'bread'): 7, ('tea', 'peas'): 6, ('bread', 'peas'): 6, ('papertowels', 'chicken'): 6, ('papertowels', 'water'): 6, ('chicken', 'water'): 6, ('chicken', 'peas'): 6}
#######################################

In [191]:
#find subsets of given size

def generateGroups(input_set,size):
    input_set = list(input_set)
    if size == 0:
        return [[]]
    new_set = []
    for index in range(0,len(input_set)):
        item = input_set[index]
        remaining = input_set[index + 1:]
        for j in generateGroups(remaining,size-1):
            new_set.append([item]+list(j))
    return new_set

def getItems(conn,transnum):
    try:
        items_list = []
        c = conn.cursor()
        c.execute("SELECT item_id FROM %s ORDER BY item_id ASC"%('transaction_'+str(transnum)))
        transaction = c.fetchall()
        for item in transaction:
            items_list.append(item[0])
        return items_list
        conn.close()
    except Error as e:
        print(e)

def getTransactions(conn,translength):
        transactions = []
        for i in range(translength):
            transaction = getItems(conn,i)
            transactions.append(transaction)
        return transactions

def scanTransactions(itemset,transactions):
    count = 0
    for i in range(len(transactions)):
        if set(itemset).issubset(transactions[i]):
            count +=1
    return count   
            
def write_rules(X,X_S, S, conf, supp, lift, num_trans):
    out_rules = ''
    out_rules += "Frequent Itemset: {}\n".format(X)
    out_rules += "    Rule: {} -> {}\n".format(list(S),X_S)
    out_rules += "    Conf: {0:2.3f}".format(conf)
    out_rules += "    Supp: {0:2.3f}".format(supp / num_trans)
    out_rules += "    Lift: {0:2.3f}\n".format(lift)
    return out_rules

#def powerset(fullset):
  #listrep = list(fullset)
  #subsets = []
  #for i in range(2**len(listrep)):
    #subset = []
    #for k in range(len(listrep)):            
      #if i & 1<<k:
        #subset.append(listrep[k])
    #subsets.append(subset)        
  #return subsets
            
def generateAssociationRules(L,min_conf,min_support,transactionsList):
    association_rules = []
    L = list(L)
    for i in range(len(L)):
        for j in range(len(L[i])):
            #print(list(L[i]))
            s = powerset(L[i])
            s.pop()
            s.remove([])
            #print(s)
            for z in s:
                S = set(z)
                X = set(L[i])
                X_S = list(X-S)
                X = list(L[i])
                S = list(z)
                sup_x = scanTransactions(X[0],transactionsList)
                sup_x_s = scanTransactions(X_S[0],transactionsList)
                if scanTransactions(S[0],transactionsList) != 0 and sup_x_s != 0:
                    conf = sup_x / scanTransactions(S[0],transactionsList)
                    lift = conf / (sup_x_s / len(transactionsList))
                else:
                    conf = 0
                    lift = 0
                if conf >= min_conf and sup_x >= min_support:
                    rule = write_rules(X,X_S,S,conf,sup_x,lift,len(transactionsList))
                    if rule not in association_rules:
                        association_rules.append(rule)
    return association_rules
            
def associationRules(L,min_confidence,min_support,transactionsList):
    association_rules = []
    L = list(L)
    for i in range(len(L)):
        for j in range(len(L[i])):
            #print(L[i][j])
            s = powerset(L[i])
            s.pop()
            s.remove([])
            #print(s)
            for k in s:
                equation = []
                #left = set(s) - set(k)
                #print(set(s))
                
                
conn = create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db")
results = apriori(getTransactions(conn,20),0.3)      
print(results)
#rules = associationRules(results,0.7,0.3,shoppinglist)

#arr = [(1,2),(3,4),(5,7)]
#print(len(arr))
#print(powerset(arr))



2.6.0
First scan results before pruning: 

bananas : 6
cabbage : 13
carrots : 8
cashews : 5
juice : 6
milk : 12
pork : 11
radish : 6
rice : 5
watermelon : 6
beer : 8
chili : 5
lettuce : 11
noodles : 9
pickles : 7
tea : 10
tomatoes : 4
bread : 9
lemons : 6
napkins : 7
papertowels : 8
tofu : 6
chicken : 9
edamame : 5
water : 6
coffee : 5
peas : 9
beef : 6
eggs : 6
##################################################

Iteration 1 
Set after pruning: 
bananas : 6
cabbage : 13
carrots : 8
juice : 6
milk : 12
pork : 11
radish : 6
watermelon : 6
beer : 8
lettuce : 11
noodles : 9
pickles : 7
tea : 10
bread : 9
lemons : 6
napkins : 7
papertowels : 8
tofu : 6
chicken : 9
water : 6
peas : 9
beef : 6
eggs : 6
##################################################

Iteration 2 
Set before pruning: 
('bananas', 'cabbage') : 3
('bananas', 'carrots') : 4
('bananas', 'juice') : 2
('bananas', 'milk') : 4
('bananas', 'pork') : 4
('bananas', 'radish') : 4
('bananas', 'watermelon') : 2
('bananas', 'beer') : 3
('

(('cabbage', 'carrots'), ('milk', 'lettuce'), ('papertowels', 'water')) : 0
(('cabbage', 'carrots'), ('milk', 'lettuce'), ('chicken', 'water')) : 0
(('cabbage', 'carrots'), ('milk', 'lettuce'), ('chicken', 'peas')) : 0
(('cabbage', 'carrots'), ('milk', 'chicken'), ('pork', 'chicken')) : 0
(('cabbage', 'carrots'), ('milk', 'chicken'), ('beer', 'lettuce')) : 0
(('cabbage', 'carrots'), ('milk', 'chicken'), ('beer', 'noodles')) : 0
(('cabbage', 'carrots'), ('milk', 'chicken'), ('lettuce', 'noodles')) : 0
(('cabbage', 'carrots'), ('milk', 'chicken'), ('lettuce', 'chicken')) : 0
(('cabbage', 'carrots'), ('milk', 'chicken'), ('tea', 'bread')) : 0
(('cabbage', 'carrots'), ('milk', 'chicken'), ('tea', 'peas')) : 0
(('cabbage', 'carrots'), ('milk', 'chicken'), ('bread', 'peas')) : 0
(('cabbage', 'carrots'), ('milk', 'chicken'), ('papertowels', 'chicken')) : 0
(('cabbage', 'carrots'), ('milk', 'chicken'), ('papertowels', 'water')) : 0
(('cabbage', 'carrots'), ('milk', 'chicken'), ('chicken', 'wat

(('cabbage', 'beer'), ('tea', 'bread'), ('bread', 'peas')) : 0
(('cabbage', 'beer'), ('tea', 'bread'), ('papertowels', 'chicken')) : 0
(('cabbage', 'beer'), ('tea', 'bread'), ('papertowels', 'water')) : 0
(('cabbage', 'beer'), ('tea', 'bread'), ('chicken', 'water')) : 0
(('cabbage', 'beer'), ('tea', 'bread'), ('chicken', 'peas')) : 0
(('cabbage', 'beer'), ('tea', 'peas'), ('bread', 'peas')) : 0
(('cabbage', 'beer'), ('tea', 'peas'), ('papertowels', 'chicken')) : 0
(('cabbage', 'beer'), ('tea', 'peas'), ('papertowels', 'water')) : 0
(('cabbage', 'beer'), ('tea', 'peas'), ('chicken', 'water')) : 0
(('cabbage', 'beer'), ('tea', 'peas'), ('chicken', 'peas')) : 0
(('cabbage', 'beer'), ('bread', 'peas'), ('papertowels', 'chicken')) : 0
(('cabbage', 'beer'), ('bread', 'peas'), ('papertowels', 'water')) : 0
(('cabbage', 'beer'), ('bread', 'peas'), ('chicken', 'water')) : 0
(('cabbage', 'beer'), ('bread', 'peas'), ('chicken', 'peas')) : 0
(('cabbage', 'beer'), ('papertowels', 'chicken'), ('pape

(('cabbage', 'chicken'), ('beer', 'noodles'), ('lettuce', 'noodles')) : 0
(('cabbage', 'chicken'), ('beer', 'noodles'), ('lettuce', 'chicken')) : 0
(('cabbage', 'chicken'), ('beer', 'noodles'), ('tea', 'bread')) : 0
(('cabbage', 'chicken'), ('beer', 'noodles'), ('tea', 'peas')) : 0
(('cabbage', 'chicken'), ('beer', 'noodles'), ('bread', 'peas')) : 0
(('cabbage', 'chicken'), ('beer', 'noodles'), ('papertowels', 'chicken')) : 0
(('cabbage', 'chicken'), ('beer', 'noodles'), ('papertowels', 'water')) : 0
(('cabbage', 'chicken'), ('beer', 'noodles'), ('chicken', 'water')) : 0
(('cabbage', 'chicken'), ('beer', 'noodles'), ('chicken', 'peas')) : 0
(('cabbage', 'chicken'), ('lettuce', 'noodles'), ('lettuce', 'chicken')) : 0
(('cabbage', 'chicken'), ('lettuce', 'noodles'), ('tea', 'bread')) : 0
(('cabbage', 'chicken'), ('lettuce', 'noodles'), ('tea', 'peas')) : 0
(('cabbage', 'chicken'), ('lettuce', 'noodles'), ('bread', 'peas')) : 0
(('cabbage', 'chicken'), ('lettuce', 'noodles'), ('papertowel

In [ ]:
#get transactions from database in list form
def getItems(conn,transnum):
    try:
        items_list = []
        c = conn.cursor()
        c.execute("SELECT * FROM %s"%('transaction_'+str(transnum)))
        transaction = c.fetchall()
        for item in transaction:
            items_list.append(item[0])
        return items_list
        conn.close()
    except Error as e:
        print(e)
   

#create hash map with collision prevention and visited flags
def createHashMap(transaction):
    hash_map = [[] for i in range(100)]
    for item in transaction:
        
        #hash item value
        h = 0
        for char in item:
            h += ord(char)
            h = h % 50
        key = h
        if hash_map[key] is None:
            hash_map[key].append(item)
        else:
            hash_map[key].append(item)
    return hash_map


#get hashes of all items in list to use for finding the support
def hashShoppingList(shoplist):
    hashlist = []
    for item in range(len(shoplist)):
        hashlist.append(getHash(shoplist[item]))
    return hashlist

        
                
#test
conn = create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db")
result_list = getItems(conn,0)
hash_map = createHashMap(result_list)
items_hash = hashShoppingList(shoppinglist)

print(hash_map)
print(items_hash)


In [195]:
def generateGroups(input_set,size):
    input_set = list(input_set)
    if size == 0:
        return [[]]
    new_set = []
    for index in range(0,len(input_set)):
        item = input_set[index]
        remaining = input_set[index + 1:]
        for j in generateGroups(remaining,size-1):
            new_set.append([item]+list(j))
    return new_set

def unionSets(input_1,input_2):
    return set(input_1).union(set(input_2))

def getItems(conn,transnum):
    try:
        items_list = []
        c = conn.cursor()
        c.execute("SELECT item_id FROM %s ORDER BY item_id ASC"%('transaction_'+str(transnum)))
        transaction = c.fetchall()
        for item in transaction:
            items_list.append(item[0])
        return items_list
        conn.close()
    except Error as e:
        print(e)

def getTransactions(conn,translength):
        transactions = []
        for i in range(translength):
            transaction = getItems(conn,i)
            transactions.append(transaction)
        return transactions

def scanTransactions(itemset,transactions):
    count = 0
    for i in range(len(transactions)):
        if set(itemset).issubset(transactions[i]):
            count +=1
    return count   

def getUnions(set1,n):
    sets = []
    for item in set1:
        if type(item) == type(''):
            item = [item]
            for i in set1:
                if type(i) == type(''):
                    i = [i]
                    if item != i:
                        u = set(item).union(set(i))
                        if u not in sets and len(u) == n:
                            sets.append(u)
        else:
            itemset = list(item)
            #print(set(itemset))
            for j in set1:
                if type(j) != type((1,2)):
                    u = set(itemset).union({j})
                    if u not in sets and len(u) == n:
                        sets.append(u)
                else:
                    itemset2 = set(list(j))
                    u = itemset2.union(itemset)
                    if u not in sets and len(u) == n:
                        sets.append(u)
                      
    return sets

def apriori(set_list,min_support):
    #initialize
    discarded = {}
    C = {}
    L = {}
    stop = False
    n = 1
    while stop == False:
        if n <= 1:
            for transaction in range(len(set_list)):
                for item in set_list[transaction]:
                    #prevent duplicate scans of same subset
                    if item not in C:
                        #prevent scans of already discarded items
                        if item not in discarded:
                            sup_count = scanTransactions({item},set_list)
                            C[item] = sup_count

            #print first scan results
            print('\nIteration {}'.format(n))
            print('C1 candidate set: \n')
            
            for i in C:
                print(i,':',C[i])
            #create L with items that meet minimum support requirement
            for item in C:
                sup_calc = C[item] / len(set_list)
                if item not in discarded:
                    if sup_calc >= min_support:
                        L[item] = C[item]
                    else:
                        discarded[item] = sup_calc
                    
            print('\nL1 set: \n')
            for i in L:
                print(i,':',L[i])
            print('#'*50)
            
            L_n = L #helps to generate sets for second iteration 
            n += 1
            if L is None:
                stop = True
                break
        elif n > 1:
            C_n = {}
            #union of the sets
            sets = []
            L_n = list(L_n)
            sets = getUnions(L_n,n)
            #print(sets)
            
            L_n_minus_1 = L_n
            #reinitialize L_n for the following iteration
            L_n = {}
            for i in range(len(sets)):
                support_count = scanTransactions(sets[i],set_list)
                C_n[tuple(sets[i])] = support_count
            if C_n == {}:
                stop = True
                break
                
            print('\nIteration {}'.format(n))
            print('C{} candidate set: \n'.format(n))
            for i in C_n:
                print(i,':',C_n[i])
            
            #calculate supports of each item in candidate set and prune
            for item in C_n:
                if item not in discarded:
                    sup_calc = C_n[item] / len(set_list)
                    if sup_calc >= min_support:
                        L_n[item] = C_n[item]
                    else:
                        discarded[item] = sup_calc
            
            print('\nL{} set: \n'.format(n))
            for i in L_n:
                print(i,':',L_n[i])
            print('#'*50)
            
            if L_n == {}:
                stop = True
                return L_n_minus_1
                break
            n += 1
            
            

conn = create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db") 
apriori(getTransactions(conn,20),0.4)



2.6.0

Iteration 1
C1 candidate set: 

bananas : 6
cabbage : 13
carrots : 8
cashews : 5
juice : 6
milk : 12
pork : 11
radish : 6
rice : 5
watermelon : 6
beer : 8
chili : 5
lettuce : 11
noodles : 9
pickles : 7
tea : 10
tomatoes : 4
bread : 9
lemons : 6
napkins : 7
papertowels : 8
tofu : 6
chicken : 9
edamame : 5
water : 6
coffee : 5
peas : 9
beef : 6
eggs : 6

L1 set: 

cabbage : 13
carrots : 8
milk : 12
pork : 11
beer : 8
lettuce : 11
noodles : 9
tea : 10
bread : 9
papertowels : 8
chicken : 9
peas : 9
##################################################

Iteration 2
C2 candidate set: 

('cabbage', 'carrots') : 6
('cabbage', 'milk') : 8
('cabbage', 'pork') : 6
('cabbage', 'beer') : 6
('cabbage', 'lettuce') : 8
('cabbage', 'noodles') : 6
('cabbage', 'tea') : 7
('cabbage', 'bread') : 5
('cabbage', 'papertowels') : 6
('cabbage', 'chicken') : 6
('cabbage', 'peas') : 5
('carrots', 'milk') : 5
('carrots', 'pork') : 5
('carrots', 'beer') : 3
('lettuce', 'carrots') : 3
('noodles', 'carrots') : 3
